Part 1

In [1]:
import numpy as np
from queue import PriorityQueue

moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]

def valid_index(arr_shape, index):
    valid = min(index) >= 0
    for s, i in zip(arr_shape, index):
        valid &= i < s
    return valid

def get_distances(filename):
    with open(filename, 'r') as f:
        data = f.read()

    tiles = np.array([list(x) for x in data.splitlines()])
    distances = np.ones_like(tiles, dtype='int') * np.inf

    start = tuple(x[0] for x in np.where(tiles == 'S'))
    distances[start] = 0
    
    todo = PriorityQueue()
    todo.put((0, start))

    while todo.queue:
        dist, current = todo.get()
        distances[current] = dist
        dist += 1

        for i, move in enumerate(moves):
            next = tuple(c + m for c, m in zip(current, move))
            if valid_index(tiles.shape, next) and any([
                i == 0 and tiles[current] in 'S|LJ' and tiles[next] in '|7F',
                i == 1 and tiles[current] in 'S|7F' and tiles[next] in '|LJ',
                i == 2 and tiles[current] in 'S-J7' and tiles[next] in '-LF',
                i == 3 and tiles[current] in 'S-LF' and tiles[next] in '-J7'
            ]) and distances[next] == np.inf:
                todo.put((dist, next))

    return distances

def pt1(filename):
    distances = get_distances(filename)
    return int(distances.max(initial = 1, where = distances != np.inf))

pt1('test.txt'), pt1('input.txt')

(8, 6725)

Part 2

In [2]:
from matplotlib.path import Path

def get_neighbours(d, index, dist=1):
    return set([
        next for move in moves 
        if valid_index(d.shape, 
            (next := tuple(c + m for c, m in zip(index, move)))
        ) and abs(d[next] - d[index]) == dist
    ])

def pt2(filename):
    d = get_distances(filename)
    start = tuple([x[0] for x in np.where(d == 0)])

    poly, next = [start], True
    while start not in poly[1:]:
        next = get_neighbours(d, poly[-1]).difference(poly[1:])
        if next:
            poly.append(next.pop())
        
    path = Path(poly)
    non_path = list(zip(*np.where(d >= np.inf)))
    return sum(path.contains_points(non_path))

pt2('test.txt'), pt2('test2.txt'), pt2('test3.txt'), pt2('input.txt')

(1, 0, 10, 383)